In [1]:
"""
adapted form 95% accurate cifar10cnn model
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import glob
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


In [2]:
#Hyper-parameters 
epoch_count=25
batch_size=32

# for training:
learning_rate = 0.001
momentum = 0.9

torch.manual_seed(143)
val_size = 1000

In [4]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
tfs=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

In [ ]:
path='/kaggle/input/breast-histopathology-images/**/*.png'
dataset=glob.glob(path,recursive=True)

In [6]:
dataset

[]

In [ ]:
print(len(train_set))
print(len(test_set))

In [ ]:

#datasplit
train_size = len(train_set) - val_size
train_data, val_data = torch.utils.data.random_split(train_set, [train_size, val_size])

ids=torch.randperm(len(train_set))

split=1000
tr_ids,val_ids=ids[split:],ids[:split]

tr_sampler=SubsetRandomSampler(tr_ids)
val_sampler=SubsetRandomSampler(val_ids)


#Dataloaders
train_loader = DataLoader(train_set, batch_size, pin_memory=True,sampler=tr_sampler)
val_loader = DataLoader(train_set, batch_size*2,pin_memory=True,sampler=val_sampler)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False,pin_memory=True)

In [ ]:

class ImgClassifier(nn.Module):
  def __init__(self):
      super(ImgClassifier, self).__init__()
      self.conv1 = nn.Conv2d(3,32,3)
      self.conv2 = nn.Conv2d(32,32,3)
    
      self.pool1 = nn.MaxPool2d(2)
      self.dout1 = nn.Dropout(0.5)
      self.conv3 = nn.Conv2d(32,64,3)
      self.conv4 = nn.Conv2d(64,64,3)

      self.pool2 = nn.MaxPool2d(2)
      self.dout2 = nn.Dropout(0.5)
      self.fc=nn.Linear(64*5*5,512)
      self.out=nn.Linear(512,10)

  def forward(self, x):
 
      x= self.conv1(x)
      x=F.relu(x)

      x= self.conv2(x)
      x=F.relu(x)

      x=self.pool1(x)
      x=self.dout1(x)

      x= self.conv3(x)
      x=F.relu(x)

      x= self.conv4(x)
      x=F.relu(x)

      x=self.pool2(x)
      x=self.dout2(x)
      #flatten all dimension except batch size
      x=torch.flatten(x,1)

      x = self.fc(x)
      x=F.relu(x)
      x=self.out(x)
      output=F.log_softmax(x)

      return output

In [ ]:
  
model=ImgClassifier()
model = model.to(device)  # put all model params on GPU.

# Create loss and optimizer
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, momentum=momentum)


In [ ]:
print(model)

In [ ]:

#variables for tracking the losses and accuracies
foo1=[]#train_loss
foo2=[]#train_accuracy
foo3=[]#valid_loss
foo4=[]#valid_accuracy
bestmodel={'epoch':  0,
        'model_state_dict': model.state_dict(),
        'loss':   0,
        'accuracy':  0
        }

In [ ]:
for epoch in range(1,epoch_count+1):
    running_loss=0.0
    running_total = 0
    running_correct = 0
    run_step = 0
   
    for i, (images, labels) in enumerate(train_loader):
        model.train()  
        
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)  
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()  # reset gradients.
        loss.backward()  # compute gradients.
        optimizer.step()  # update parameters.

        running_loss += loss.item()
        running_total += labels.size(0)

        with torch.no_grad():
            _, predicted = outputs.max(1)
        running_correct += (predicted == labels).sum().item()
        run_step += 1
        if i %  500== 0:
            # check accuracy.
            print(f'epoch: {epoch}, steps: {i}, '
                  f'train_loss: {running_loss / run_step :.3f}, '
                  f'running_acc: {100 * running_correct / running_total:.1f} %')
            foo1.append(running_loss/run_step)
            foo2.append(100 * running_correct / running_total)
            running_loss = 0.0
            running_total = 0
            running_correct = 0
            run_step = 0

    # validation
    best_val_acc=[0,0]
    correct = 0
    val_loss=0
    total = 0
    counter=0
    model.eval()
    
    with torch.no_grad():
        for data in val_loader:
            counter+=1
            images,labels=data
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            val_loss += loss_fn(outputs,labels).item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_acc = 100 * correct / total

    if 100 * correct / total >= bestmodel['accuracy']: 
      bestmodel={'epoch':  epoch,
        'model_state_dict': model.state_dict(),
        'loss':   val_loss,
        'accuracy':   val_acc
        }
    print(f'Validation accuracy: {100 * correct / total: .3f}%   ,'
              f'Validation loss:{val_loss/counter:.3f}')
    foo3.append(val_loss/counter)
    foo4.append(100 * correct / total)

print('Finished Training')

In [ ]:
lastmodel={'epoch':  epoch_count,
        'model_state_dict': model.state_dict(),
        'loss':   val_loss,
        'accuracy':   val_acc
        }

torch.save(lastmodel,'lastmodel.pth')
torch.save(bestmodel,'bestmodel.pth')

In [ ]:
#evaluation
with torch.no_grad():
    correct = 0
    total = 0
    model.eval() 
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs=model(images)
        _, predicted = outputs.max(dim=1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    test_acc = 100 * correct / total
    print(f'Test accuracy: {test_acc} %')
    print(f'Test error rate: {100 - 100 * correct / total: .2f} %')


In [ ]:
#plotting the losses and accuracies
x=np.linspace(0,epoch_count,epoch_count*4)
x1=np.arange(0,epoch_count)

plt.figure(figsize=(10,7))
plt.plot(
    x,foo1,'g-',label='training loss'
)
plt.plot(
    x1,foo3,'b-',label='validation loss'
)
plt.xlabel("Epoch count")
plt.ylabel("Magnitude")
plt.title("Training and Validation Losses")
plt.legend()
plt.savefig('val_train_loss.png',dpi=300)
plt.show()

plt.figure(figsize=(10,7))
plt.plot(
    x,foo2,'r-',label='training accuracy'
)
plt.plot(
    x1,foo4,'b-',label='validation accuracy'
)
plt.xlabel("Epoch count")
plt.ylabel("Magnitude")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.savefig('val_train_acccuracy.png',dpi=300)
plt.show()